# Optimise block diagonal matrix multiplication

Optimise computing an operation of multiplying a block diagonal matrix with a vector, which is the bottleneck in the current `SkillsGPMAP.fit()` step.

## Setup.

In [1]:
import os
if 'src' not in os.listdir():
    os.chdir("..")

In [65]:
import gzip
import importlib
import json
import pickle

import numpy as np
import pandas as pd
import scipy.linalg

import numba

from src import load
import src.models.gp
import src.stats
import src.statics

In [3]:
pd.set_option('max_rows', 6)
pd.set_option('max_columns', 50)

## Load data.

In [4]:
with gzip.open('data/raw/premium_matches.2019-08-31.json.gz', 'rb') as fh:
    premium_matches = load.matches_json_to_df(json.load(fh)['data'])
    prm_m = load.MatchDF(premium_matches)

In [5]:
premium_pro_model = src.models.gp.SkillsGPMAP(
    None,
    None,
    prm_m.players_mat,
    prm_m.df.startTimestamp,
    prm_m.df.radiantVictory,
    'exponential',
    {'scale': 1.25 * 365 * 24 * 60 * 60 * 1000},
    radi_prior_sd=3.0,
    logistic_scale=3.0)

In [6]:
minus_inv_cov_mats = [np.ascontiguousarray(-scipy.linalg.inv(x))
                      for x in premium_pro_model.cov_mats]
radi_adv_cov_mat = np.full((1, 1),
                           -1 / (premium_pro_model.radi_prior_sd ** 2))
minus_inv_cov_mat_list = minus_inv_cov_mats + [radi_adv_cov_mat]

In [7]:
minus_inv_cov_mat_list[0].flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [8]:
radi_adv_cov_mat.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [9]:
size = sum([x.shape[0] for x in minus_inv_cov_mat_list])
size

50001

In [10]:
p = np.repeat(0.0, size)
p

array([0., 0., 0., ..., 0., 0., 0.])

## Timings

### Current implementation using `scipy.sparse.block_diag()`.

In [11]:
minus_inv_cov_mat = scipy.sparse.block_diag(minus_inv_cov_mat_list,
                                            format='csr')

In [12]:
%timeit minus_inv_cov_mat @ p

9.92 ms ± 453 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### By manually splitting and concatenating *p*

In [13]:
breaks = np.cumsum([x.shape[0] for x in minus_inv_cov_mat_list])
breaks

array([  110,   150,   298,   349,   410,   423,   628,   655,   959,
        1108,  1184,  1405,  1415,  1581,  1584,  1829,  1850,  1851,
        1883,  1904,  1928,  1975,  1977,  1983,  2254,  2364,  2415,
        2670,  2723,  2867,  2873,  3013,  3034,  3045,  3335,  3336,
        3341,  3414,  3421,  3425,  3458,  3462,  3566,  3706,  3713,
        4010,  4014,  4018,  4267,  4279,  4384,  4387,  4566,  4571,
        4621,  4842,  4848,  4869,  4899,  4905,  4907,  5159,  5162,
        5202,  5203,  5210,  5265,  5323,  5342,  5373,  5376,  5499,
        5527,  5535,  5586,  5587,  5589,  5590,  5613,  5678,  5687,
        5718,  5800,  5824,  5832,  5846,  6094,  6095,  6105,  6106,
        6426,  6427,  6446,  6448,  6482,  6518,  6770,  6776,  6779,
        6782,  6790,  6802,  6998,  7271,  7366,  7534,  7535,  7577,
        7579,  7584,  7591,  7774,  7781,  8012,  8189,  8214,  8269,
        8291,  8297,  8304,  8316,  8320,  8440,  8453,  8778,  8995,
        9139,  9162,

In [14]:
def block_diag_list_mult(mat_list, split_p):
    out_vals = []
    for mm, pp in zip(mat_list, split_p):
        out_vals.append(mm @ pp)
    return np.concatenate(out_vals)

In [15]:
split_p = np.split(p, breaks[:-1])
%timeit block_diag_list_mult(minus_inv_cov_mat_list, split_p)

11 ms ± 3.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### By using numba

In [16]:
@numba.jit(nopython=True)
def block_diag_list_mult_jit(mat_list, p, breaks):
    out_vec = np.zeros(p.size)
    prev_i = 0
    for i in range(len(mat_list)):
        m = mat_list[i]
        b = breaks[i]
        out_vec[prev_i:b] = m @ p[prev_i:b]
        prev_i = b
    return out_vec

In [57]:
sum([x.shape[0] for x in minus_inv_cov_mat_list[:700]])

42777

In [62]:
%timeit block_diag_list_mult_jit(tuple(minus_inv_cov_mat_list), p, breaks)

15.5 ms ± 2.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### By using numba with fastmath=True

In [69]:
@numba.jit(nopython=True, fastmath=True)
def block_diag_list_mult_jit(mat_list, p, breaks):
    out_vec = np.zeros(p.size)
    prev_i = 0
    for i in range(len(mat_list)):
        m = mat_list[i]
        b = breaks[i]
        out_vec[prev_i:b] = m @ p[prev_i:b]
        prev_i = b
    return out_vec

In [71]:
%timeit block_diag_list_mult_jit(tuple(minus_inv_cov_mat_list), p, breaks)

6.64 ms ± 444 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Try `SkillsGPMAP.fit()` with numba.

In [72]:
importlib.reload(src.models.gp)
premium_pro_model = src.models.gp.SkillsGPMAP(
    None,
    None,
    prm_m.players_mat,
    prm_m.df.startTimestamp,
    prm_m.df.radiantVictory,
    'exponential',
    {'scale': 1.25 * 365 * 24 * 60 * 60 * 1000},
    radi_prior_sd=3.0,
    logistic_scale=3.0)

In [73]:
premium_pro_model.fit()